Data is fairly well-formatted to begin with, so we'll just:
- Merge control and pilot data into one df
- Ensure that the 'NULL' is saved as NaN, code other NA responses as NaN
- Set data types for dates appropriately
- Clean up some values like 01/03/2company3
- set appropriate data types, like for Volt Flag
- Set Columns for treatment/control

This should set up our quant text analysis and segmented modelling.

#### Import Libraries and Data

In [37]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [38]:
# Read both sheets from the Excel file
file_path = '../data/raw/CDS_25_Task2.xlsx'

# Read control and pilot data
control_df = pd.read_excel(file_path, sheet_name='C Control', dtype={'TO_CHAR': 'object'})
pilot_df = pd.read_excel(file_path, sheet_name='C Pilot', dtype={'TO_CHAR': 'object'})

print("Control data shape:", control_df.shape)
print("Pilot data shape:", pilot_df.shape)
print("\nControl columns:", control_df.columns.tolist())
print("\nPilot columns:", pilot_df.columns.tolist())

# Display first few rows to understand structure
print("\n--- Control Data Sample ---")
print(control_df.head())
print("\n--- Pilot Data Sample ---")
print(pilot_df.head())

Control data shape: (380, 13)
Pilot data shape: (267, 13)

Control columns: ['COLUMN_4', 'VOLT_FLAG', 'SURVEY_ID', 'SCORE', 'LTR_COMMENT', 'PRIMARY_REASON', 'TO_CHAR', 'CONNECTION_TIME', 'SALES_PERSON_SAT', 'SALES_FRIENDLY_SAT', 'COMMINICATION_SAT', 'FIRST_BILL_SAT', 'AGENT_KNOWLEDGE']

Pilot columns: ['COLUMN_4', 'VOLT_FLAG', 'SURVEY_ID', 'SCORE', 'LTR_COMMENT', 'PRIMARY_REASON', 'TO_CHAR', 'CONNECTION_TIME', 'SALES_PERSON_SAT', 'SALES_FRIENDLY_SAT', 'COMMINICATION_SAT', 'FIRST_BILL_SAT', 'AGENT_KNOWLEDGE']

--- Control Data Sample ---
  COLUMN_4 VOLT_FLAG  SURVEY_ID  SCORE  \
0  control       NaN  351124494     10   
1  control       yes  351727144     10   
2  control       yes  351645749     10   
3  control       yes  351707395     10   
4  control       NaN  351119086     10   

                                         LTR_COMMENT  \
0  Conservations with your staff over the phone w...   
1  I spoke to tony and he was lovely, he talked m...   
2                                   

In [39]:
#Basic data inspection
print("Control data shape:", control_df.shape)
print("Pilot data shape:", pilot_df.shape)
print("\nColumn names match:", list(control_df.columns) == list(pilot_df.columns))

Control data shape: (380, 13)
Pilot data shape: (267, 13)

Column names match: True


#### Merge datasets and mutate new variables

In [40]:
# Merge the datasets
df = pd.concat([control_df, pilot_df], ignore_index=True)

# Rename COLUMN_4 to group
df.rename(columns={'COLUMN_4': 'GROUP'}, inplace=True)

print(f"Combined dataset shape: {df.shape}")
print(f"Treatment group distribution:\n{df['GROUP'].value_counts()}")

Combined dataset shape: (647, 13)
Treatment group distribution:
GROUP
control    380
pilot      267
Name: count, dtype: int64


In [41]:
# Add volt flag binary, set as integer where 'yes' is 1 and others are 0
df['VOLT_FLAG_BINARY'] = df['VOLT_FLAG'].map({'yes': 1}).fillna(0).astype(int)

# Add treatment flag binary, set as integer where 'yes' is 1 and others are 0
df['TREATMENT_BINARY'] = df['GROUP'].map({'pilot': 1}).fillna(0).astype(int)

# See if it worked
print(f"Volt flag binary distribution:\n{df['VOLT_FLAG_BINARY'].value_counts()}")
print(f"Treatment flag binary distribution:\n{df['TREATMENT_BINARY'].value_counts()}")


Volt flag binary distribution:
VOLT_FLAG_BINARY
0    368
1    279
Name: count, dtype: int64
Treatment flag binary distribution:
TREATMENT_BINARY
0    380
1    267
Name: count, dtype: int64


#### Dealing with missing values

In [42]:
print(f"Missing values in each column:\n{df.isnull().sum()}")

Missing values in each column:
GROUP                   0
VOLT_FLAG             368
SURVEY_ID               0
SCORE                   0
LTR_COMMENT           165
PRIMARY_REASON        286
TO_CHAR                 0
CONNECTION_TIME        20
SALES_PERSON_SAT       55
SALES_FRIENDLY_SAT     30
COMMINICATION_SAT      23
FIRST_BILL_SAT         23
AGENT_KNOWLEDGE        17
VOLT_FLAG_BINARY        0
TREATMENT_BINARY        0
dtype: int64


Seems good as is, NA's correctly encoded

#### Clean Up Dates

This was done manually in excel, as it was the simplest approach, modified original data to get rid of the 'company' string in dates

In [43]:
# Rename TO_CHAR to MONTH
df.rename(columns={'TO_CHAR': 'MONTH'}, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647 entries, 0 to 646
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   GROUP               647 non-null    object 
 1   VOLT_FLAG           279 non-null    object 
 2   SURVEY_ID           647 non-null    object 
 3   SCORE               647 non-null    int64  
 4   LTR_COMMENT         482 non-null    object 
 5   PRIMARY_REASON      361 non-null    object 
 6   MONTH               647 non-null    object 
 7   CONNECTION_TIME     627 non-null    float64
 8   SALES_PERSON_SAT    592 non-null    float64
 9   SALES_FRIENDLY_SAT  617 non-null    float64
 10  COMMINICATION_SAT   624 non-null    float64
 11  FIRST_BILL_SAT      624 non-null    float64
 12  AGENT_KNOWLEDGE     630 non-null    float64
 13  VOLT_FLAG_BINARY    647 non-null    int64  
 14  TREATMENT_BINARY    647 non-null    int64  
dtypes: float64(6), int64(3), object(6)
memory usage: 75.9+ KB

In [44]:
# print unique values in MONTH column
print(f"Unique values in MONTH column:\n{df['MONTH'].unique()}")

Unique values in MONTH column:
[datetime.datetime(2023, 2, 1, 0, 0) datetime.datetime(2023, 3, 1, 0, 0)
 datetime.datetime(2023, 4, 1, 0, 0) datetime.datetime(2023, 5, 1, 0, 0)
 datetime.datetime(2023, 6, 1, 0, 0)]


#### Check data types to confirm

In [45]:
# convert columns in indexes 7-12 to int64
for col in df.columns[7:13]:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

# convert DATE column to datetime
df['MONTH'] = pd.to_datetime(df['MONTH'], errors='coerce')
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647 entries, 0 to 646
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   GROUP               647 non-null    object        
 1   VOLT_FLAG           279 non-null    object        
 2   SURVEY_ID           647 non-null    object        
 3   SCORE               647 non-null    int64         
 4   LTR_COMMENT         482 non-null    object        
 5   PRIMARY_REASON      361 non-null    object        
 6   MONTH               647 non-null    datetime64[ns]
 7   CONNECTION_TIME     627 non-null    Int64         
 8   SALES_PERSON_SAT    592 non-null    Int64         
 9   SALES_FRIENDLY_SAT  617 non-null    Int64         
 10  COMMINICATION_SAT   624 non-null    Int64         
 11  FIRST_BILL_SAT      624 non-null    Int64         
 12  AGENT_KNOWLEDGE     630 non-null    Int64         
 13  VOLT_FLAG_BINARY    647 non-null    int64         

#### Save our cleaned data

In [46]:
# Save cleaned data for next steps as pickle and CSV
df.to_pickle('../data/processed/cleaned_call_script_data.pkl')
df.to_csv('../data/processed/cleaned_call_script_data.csv', index=False)

#### Add a codebook (.md) for all the variables

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647 entries, 0 to 646
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   GROUP               647 non-null    object        
 1   VOLT_FLAG           279 non-null    object        
 2   SURVEY_ID           647 non-null    object        
 3   SCORE               647 non-null    int64         
 4   LTR_COMMENT         482 non-null    object        
 5   PRIMARY_REASON      361 non-null    object        
 6   MONTH               647 non-null    datetime64[ns]
 7   CONNECTION_TIME     627 non-null    Int64         
 8   SALES_PERSON_SAT    592 non-null    Int64         
 9   SALES_FRIENDLY_SAT  617 non-null    Int64         
 10  COMMINICATION_SAT   624 non-null    Int64         
 11  FIRST_BILL_SAT      624 non-null    Int64         
 12  AGENT_KNOWLEDGE     630 non-null    Int64         
 13  VOLT_FLAG_BINARY    647 non-null    int64         

#### Data segmentation/filtering

In [ ]:
# Getting rid of february data
df_no_feb = df[df['MONTH'].dt.month != 2]
df_no_feb.info()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 582 entries, 45 to 646
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   GROUP               582 non-null    object        
 1   VOLT_FLAG           241 non-null    object        
 2   SURVEY_ID           582 non-null    object        
 3   SCORE               582 non-null    int64         
 4   LTR_COMMENT         430 non-null    object        
 5   PRIMARY_REASON      326 non-null    object        
 6   MONTH               582 non-null    datetime64[ns]
 7   CONNECTION_TIME     565 non-null    Int64         
 8   SALES_PERSON_SAT    534 non-null    Int64         
 9   SALES_FRIENDLY_SAT  558 non-null    Int64         
 10  COMMINICATION_SAT   562 non-null    Int64         
 11  FIRST_BILL_SAT      562 non-null    Int64         
 12  AGENT_KNOWLEDGE     568 non-null    Int64         
 13  VOLT_FLAG_BINARY    582 non-null    int64         
 14

In [53]:
# Segmenting into non-VOLT
df_non_volt = df[df['VOLT_FLAG_BINARY'] == 0]
print(f"Non-VOLT data shape: {df_non_volt.shape}")
df_non_volt.info()

# Segmenting into VOLT
df_volt = df[df['VOLT_FLAG_BINARY'] == 1]
print(f"VOLT data shape: {df_volt.shape}")
df_volt.info()

# Segmenting into treatment group
df_treatment = df[df['TREATMENT_BINARY'] == 1]
print(f"Treatment group data shape: {df_treatment.shape}")
df_treatment.info()



Non-VOLT data shape: (368, 15)
<class 'pandas.core.frame.DataFrame'>
Index: 368 entries, 0 to 646
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   GROUP               368 non-null    object        
 1   VOLT_FLAG           0 non-null      object        
 2   SURVEY_ID           368 non-null    object        
 3   SCORE               368 non-null    int64         
 4   LTR_COMMENT         281 non-null    object        
 5   PRIMARY_REASON      211 non-null    object        
 6   MONTH               368 non-null    datetime64[ns]
 7   CONNECTION_TIME     359 non-null    Int64         
 8   SALES_PERSON_SAT    338 non-null    Int64         
 9   SALES_FRIENDLY_SAT  354 non-null    Int64         
 10  COMMINICATION_SAT   356 non-null    Int64         
 11  FIRST_BILL_SAT      357 non-null    Int64         
 12  AGENT_KNOWLEDGE     357 non-null    Int64         
 13  VOLT_FLAG_BINARY    368 